# Summarize Runs
Summarize the runs that have been performed to date

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from datetime import datetime
from glob import glob
import pandas as pd
import json
import re
import os

## Find the Result Directories
They each contain a `runtime.log` file

In [2]:
models = glob(os.path.join('runs', '**', 'results.json'))

In [3]:
def load_models(log_path):
    """Get the information from the run in
    
    Args:
        log_path (str): Path to the runtime log
    """
    
    # Store the path
    run_dir = os.path.dirname(log_path)
    path_name = os.path.basename(run_dir)
    output = {
        'path': run_dir,
        'start_time': datetime.strptime(path_name.split("-", 3)[-1], "%d%m%y-%H%M%S"),
    }
    
    # Get the run parameters
    with open(os.path.join(run_dir, 'params.json')) as fp:
        output.update(json.load(fp))
    for p in ['redishost', 'redisport']:
        del output[p]
        
    # Count the number of workers
    #  TODO (wardlt): Divide up by manager type
    for m in glob(os.path.join(run_dir, '**', 'manager.log'), recursive=True):
        workers = 0
        with open(m) as fp:
            workers += int(re.findall('Manager will spawn (\d+) workers', fp.read(), re.MULTILINE)[0])
    output['worker_count'] = workers
        
    # Get the number evaluated
    with open(os.path.join(run_dir, 'results.json')) as fp:
        output['n_evals'] = len(fp.readlines())
    
    return output

In [4]:
results = pd.DataFrame(load_models(m) for m in models)
results

,path,start_time,num_guesses,num_parallel,dim,opt_delay,runtime,runtime_var,file,worker_count,n_evals
0,runs/streaming-N100-P4-050920-153723,2020-09-05 15:37:23,100,4,4,1.5,2.0,0.5,streaming.py,4,100
1,runs/streaming-N20-P4-050920-154259,2020-09-05 15:42:59,20,4,4,4.0,2.0,0.5,streaming.py,4,20
2,runs/interleaved-N20-P4-070920-193000,2020-09-07 19:30:00,20,4,4,4.0,2.0,0.5,interleaved.py,1,19
3,runs/batch-N100-P4-130820-135840,2020-08-13 13:58:40,100,4,4,NaN,10.0,4.0,batch.py,4,100
4,runs/batch-N20-P2-130820-144940,2020-08-13 14:49:40,20,2,4,NaN,10.0,4.0,batch.py,2,20
5,runs/streaming-N100-P4-050920-145955,2020-09-05 14:59:55,100,4,4,NaN,2.0,1.0,streaming.py,4,100
6,runs/batch-N20-P4-210720-173917,2020-07-21 17:39:17,20,4,4,NaN,5.0,3.0,batch.py,4,20


In [5]:
results.to_csv('run_data.csv', index=False)